In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
data = pd.read_csv('/content/Heart Disease/balanced_data_42K.csv')
data.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,60,1,164,92.0,130,90,2,1,0,0,1,0
1,58,1,163,60.0,120,80,1,1,0,0,1,1
2,58,1,147,57.0,130,80,1,1,0,0,1,1
3,50,2,169,89.0,188,105,3,2,0,0,1,1
4,63,1,155,59.0,140,70,2,1,0,0,1,1


In [9]:
# Set random seeds for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

In [10]:
# Split the dataset
X = data.drop('cardio', axis=1)
y = data['cardio']

In [16]:
X

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,60,1,164,92.0,130,90,2,1,0,0,1
1,58,1,163,60.0,120,80,1,1,0,0,1
2,58,1,147,57.0,130,80,1,1,0,0,1
3,50,2,169,89.0,188,105,3,2,0,0,1
4,63,1,155,59.0,140,70,2,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
42495,62,1,158,54.0,110,70,1,1,0,0,1
42496,42,2,177,95.0,120,90,2,1,0,0,1
42497,59,2,165,71.0,130,70,1,3,0,0,1
42498,50,1,165,68.0,120,80,1,1,0,0,0


In [18]:
y

0        0
1        1
2        1
3        1
4        1
        ..
42495    0
42496    0
42497    0
42498    1
42499    0
Name: cardio, Length: 42500, dtype: int64

In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

In [12]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
#  ANN model with fixed seeds
ANN = keras.Sequential([
    layers.Input(shape=(11,)),
    layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value)),
    layers.Dropout(0.5), 
    layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value)),
    layers.Dropout(0.5),  
    layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=seed_value))
])

In [14]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the model
ANN.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
# Train the model with early stopping
ANN.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Make predictions
y_pred = ANN.predict(X_test)
y_pred = (y_pred > 0.5)  # Convert probabilities to binary values

# Calculate evaluation metrics
accuracy_ANN = accuracy_score(y_test, y_pred)
precision_ANN = precision_score(y_test, y_pred)
recall_ANN = recall_score(y_test, y_pred)
f1_ANN = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score


y_pred_prob_ANN = ANN.predict(X_test).ravel()
auc_roc_ANN = roc_auc_score(y_test, y_pred_prob_ANN)
print(f'ANN AUC: {auc_roc_ANN}')


print(f'Accuracy: {accuracy_ANN:.2f}')
print(f'Precision: {precision_ANN:.2f}')
print(f'Recall: {recall_ANN:.2f}')
print(f'F1 Score: {f1_ANN:.2f}')
print('Confusion Matrix:')
print(confusion)

Epoch 1/100
1076/1076 [==============================] - 8s 4ms/step - loss: 0.6516 - accuracy: 0.6213 - val_loss: 0.6296 - val_accuracy: 0.6554
Epoch 2/100
1076/1076 [==============================] - 4s 3ms/step - loss: 0.6184 - accuracy: 0.6717 - val_loss: 0.6016 - val_accuracy: 0.6863
Epoch 3/100
1076/1076 [==============================] - 4s 3ms/step - loss: 0.5961 - accuracy: 0.7009 - val_loss: 0.5753 - val_accuracy: 0.7176
Epoch 4/100
1076/1076 [==============================] - 4s 3ms/step - loss: 0.5868 - accuracy: 0.7127 - val_loss: 0.5685 - val_accuracy: 0.7221
Epoch 5/100
1076/1076 [==============================] - 4s 3ms/step - loss: 0.5755 - accuracy: 0.7178 - val_loss: 0.5625 - val_accuracy: 0.7221
Epoch 6/100
1076/1076 [==============================] - 4s 3ms/step - loss: 0.5703 - accuracy: 0.7227 - val_loss: 0.5604 - val_accuracy: 0.7268
Epoch 7/100
1076/1076 [==============================] - 4s 3ms/step - loss: 0.5659 - accuracy: 0.7256 - val_loss: 0.5607 - val_ac